El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import time

import catboost as cb
from catboost import CatBoostRegressor

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
data = pd.read_csv('/datasets/car_data.csv')

In [3]:
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [5]:
data.describe()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [6]:
# Se sustituyen los valores en la columna NotRepaired (No, Yes) por valores binarios 0, 1 para que la data sea más manejable
# a la hora del entrenamiento, así como también se tratan los NaN's de la misma columna sustituyendo por 0

data['NotRepaired'] = data['NotRepaired'].replace('yes', 1)
data['NotRepaired'] = data['NotRepaired'].replace('no', 0)
data['NotRepaired'] = data['NotRepaired'].fillna(0)

data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,0.0,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,1.0,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,0.0,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,0.0,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,0.0,31/03/2016 00:00,0,60437,06/04/2016 10:17


In [7]:
# Se verifica si hay valores nulos, y que porcentaje ocupan, para saber como se tratarán

(data.isnull().sum() / len(data)*100).round(2).sort_values(ascending=False)

VehicleType          10.58
FuelType              9.28
Gearbox               5.60
Model                 5.56
DateCrawled           0.00
Price                 0.00
RegistrationYear      0.00
Power                 0.00
Mileage               0.00
RegistrationMonth     0.00
Brand                 0.00
NotRepaired           0.00
DateCreated           0.00
NumberOfPictures      0.00
PostalCode            0.00
LastSeen              0.00
dtype: float64

Podemos visualizar que las columnas donde hay valores ausentes son columnas con valores cualitativos que no afectan el entrenamiento del modelo, por lo tanto, no se imputan los valores ausentes.

In [8]:
# Se verifica si hay valores duplicados y cuantos hay

data.duplicated().sum()

291

Se encuentran algunos valores duplicados y para no alterar el entrenamiento del modelo se decide eliminar los valores duplicados. Adicionalmente, son muy pocos valores ausentes comparados con el total de filas del dataframe.

In [9]:
data = data.drop_duplicates()

In [10]:
data.duplicated().sum()

0

In [11]:
data['RegistrationYear'].unique()

array([1993, 2011, 2004, 2001, 2008, 1995, 1980, 2014, 1998, 2005, 1910,
       2016, 2007, 2009, 2002, 2018, 1997, 1990, 2017, 1981, 2003, 1994,
       1991, 1984, 2006, 1999, 2012, 2010, 2000, 1992, 2013, 1996, 1985,
       1989, 2015, 1982, 1976, 1983, 1973, 1111, 1969, 1971, 1987, 1986,
       1988, 1970, 1965, 1945, 1925, 1974, 1979, 1955, 1978, 1972, 1968,
       1977, 1961, 1960, 1966, 1975, 1963, 1964, 5000, 1954, 1958, 1967,
       1959, 9999, 1956, 3200, 1000, 1941, 8888, 1500, 2200, 4100, 1962,
       1929, 1957, 1940, 3000, 2066, 1949, 2019, 1937, 1951, 1800, 1953,
       1234, 8000, 5300, 9000, 2900, 6000, 5900, 5911, 1933, 1400, 1950,
       4000, 1948, 1952, 1200, 8500, 1932, 1255, 3700, 3800, 4800, 1942,
       7000, 1935, 1936, 6500, 1923, 2290, 2500, 1930, 1001, 9450, 1944,
       1943, 1934, 1938, 1688, 2800, 1253, 1928, 1919, 5555, 5600, 1600,
       2222, 1039, 9996, 1300, 8455, 1931, 1915, 4500, 1920, 1602, 7800,
       9229, 1947, 1927, 7100, 8200, 1946, 7500, 35

In [12]:
data=data[(data['RegistrationYear'] <= 2025) & (data['RegistrationYear'] >= 1910)]
data['RegistrationYear'].unique()

array([1993, 2011, 2004, 2001, 2008, 1995, 1980, 2014, 1998, 2005, 1910,
       2016, 2007, 2009, 2002, 2018, 1997, 1990, 2017, 1981, 2003, 1994,
       1991, 1984, 2006, 1999, 2012, 2010, 2000, 1992, 2013, 1996, 1985,
       1989, 2015, 1982, 1976, 1983, 1973, 1969, 1971, 1987, 1986, 1988,
       1970, 1965, 1945, 1925, 1974, 1979, 1955, 1978, 1972, 1968, 1977,
       1961, 1960, 1966, 1975, 1963, 1964, 1954, 1958, 1967, 1959, 1956,
       1941, 1962, 1929, 1957, 1940, 1949, 2019, 1937, 1951, 1953, 1933,
       1950, 1948, 1952, 1932, 1942, 1935, 1936, 1923, 1930, 1944, 1943,
       1934, 1938, 1928, 1919, 1931, 1915, 1920, 1947, 1927, 1946])

In [13]:
data.reset_index(inplace=True)

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353907 entries, 0 to 353906
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   index              353907 non-null  int64  
 1   DateCrawled        353907 non-null  object 
 2   Price              353907 non-null  int64  
 3   VehicleType        316594 non-null  object 
 4   RegistrationYear   353907 non-null  int64  
 5   Gearbox            334215 non-null  object 
 6   Power              353907 non-null  int64  
 7   Model              334281 non-null  object 
 8   Mileage            353907 non-null  int64  
 9   RegistrationMonth  353907 non-null  int64  
 10  FuelType           321146 non-null  object 
 11  Brand              353907 non-null  object 
 12  NotRepaired        353907 non-null  float64
 13  DateCreated        353907 non-null  object 
 14  NumberOfPictures   353907 non-null  int64  
 15  PostalCode         353907 non-null  int64  
 16  La

## Entrenamiento del modelo 

In [15]:
# Codificación OHE

print(data['Gearbox'].unique())
print(data['FuelType'].unique())

['manual' 'auto' nan]
['petrol' 'gasoline' nan 'lpg' 'other' 'hybrid' 'cng' 'electric']


In [16]:
data_ohe = pd.get_dummies(data, columns=['Gearbox', 'FuelType'], drop_first=True, dtype = int)

data_ohe.sample(5)

,index,DateCrawled,Price,VehicleType,RegistrationYear,Power,Model,Mileage,RegistrationMonth,Brand,...,NumberOfPictures,PostalCode,LastSeen,Gearbox_manual,FuelType_electric,FuelType_gasoline,FuelType_hybrid,FuelType_lpg,FuelType_other,FuelType_petrol
235452,235707,30/03/2016 19:54,16000,bus,2012,140,touran,100000,1,volkswagen,...,0,46342,01/04/2016 14:46,0,0,1,0,0,0,0
254618,254901,28/03/2016 22:57,1950,suv,1994,116,other,150000,5,opel,...,0,69234,29/03/2016 06:43,1,0,0,0,0,0,1
30800,30817,11/03/2016 21:56,3800,NaN,2005,0,s_klasse,150000,0,mercedes_benz,...,0,31319,06/04/2016 22:16,0,0,0,0,0,0,1
185999,186182,12/03/2016 17:44,5000,small,2009,68,aygo,40000,8,toyota,...,0,57489,07/04/2016 08:45,1,0,0,0,0,0,1
10706,10707,20/03/2016 12:42,2350,small,2006,0,1_reihe,100000,7,peugeot,...,0,41462,20/03/2016 12:42,1,0,0,0,0,0,1


In [17]:
features = data_ohe[['RegistrationYear','Power','Mileage','NotRepaired', 'Gearbox_manual', 'FuelType_electric', 'FuelType_gasoline', 'FuelType_hybrid', 'FuelType_lpg', 'FuelType_other', 'FuelType_petrol']]
target = data['Price']

In [18]:
# Se divide el dataframe en entrenamiento, prueba y validación

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=42)
features_train, features_val, target_train, target_val = train_test_split(features_train, target_train, test_size=0.25, random_state=12345)

In [19]:
# Regresión Lineal

%time
#start_time=time.time()

lr = LinearRegression()
lr.fit(features_train, target_train)
pred_lr = lr.predict(features_val)

#end_time=time.time()
#total_time=end_time - start_time
#print(total_time)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.72 µs


In [20]:
# Arbol de decisión

%time

model = DecisionTreeRegressor(random_state=42)
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 10, 20],
    'min_samples_leaf': [1, 5, 10]
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(features_train, target_train)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

pred_dt = best_model.predict(features_val)

CPU times: user 9 µs, sys: 3 µs, total: 12 µs
Wall time: 4.77 µs


In [21]:
# LightGBM

%time

lgbm = LGBMRegressor(random_state=42)

# Se define la matriz de hiperparámetros a probar
param_grid_lgbm = {
    'n_estimators': [100, 200],
    'learning_rate': [0.1, 0.01],
}

grid_search_lgbm = GridSearchCV(lgbm, param_grid_lgbm, cv=3, scoring='neg_mean_squared_error')
grid_search_lgbm.fit(features_train, target_train)

pred_lgbm = grid_search_lgbm.predict(features_val)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs


In [22]:
# CatBoosRegressor

%time

model = CatBoostRegressor(iterations=500, learning_rate=0.1, depth=6, verbose=0)
model.fit(features_train, target_train) 

pred_cboost = model.predict(features_val)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


## Análisis del modelo

In [23]:
# Regresión Lineal

rmse_lr = mean_squared_error(target_test, pred_lr,squared=False)
print(f'RMSE para Regresión Lineal: {rmse_lr}')

RMSE para Regresión Lineal: 5250.645420451536


In [24]:
# Arbol de decisión

rmse_dt = mean_squared_error(target_test, pred_dt, squared=False)
print(f'RMSE para Bosque Aleatorio: {rmse_dt}')

RMSE para Bosque Aleatorio: 6095.267654622543


In [25]:
# LightGBM

rmse_lgbm = mean_squared_error(target_test, pred_lgbm, squared=False)
print(f'RMSE para LightGBM: {rmse_lgbm}')

RMSE para LightGBM: 6027.631268773323


In [26]:
# CatBoostRegressor

rmse_cboost = mean_squared_error(target_test, pred_cboost, squared=False)

print(f"RMSE para CatBoostRegressor: {rmse_cboost}")

RMSE para CatBoostRegressor: 6022.354636161281


Como podemos visualizar, en el apartado de entrenamiento de los modelos, tanto la regresión lineal como catboostregressor son los que lograron menos tiempo de entrenamiento, seguido por lighgbm y por último el modelo de árboles de decisión. La diferencia es considerable entre los diferentes modelos, sin embargo en donde destaca el modelo lightgbm en en su calidad, ya que fue la que mejor respuesta arrojo al realizar el análisis. Hasta este punto, el model lightgbm parece no superar al modelo de regresión lineal, por lo que se procederá a realizar un análisis de ambos en el conjunto de test para comprobarlo.

## Evaluación del mejor modelo

In [27]:
# Regresión lineal

pred_lr = lr.predict(features_val)
rmse_lr = mean_squared_error(target_test, pred_lr,squared=False)
print(f'RMSE para Regresión Lineal: {rmse_lr}')

RMSE para Regresión Lineal: 5250.645420451536


In [28]:
#LightGBM

pred_lgbm = grid_search_lgbm.predict(features_test)
rmse_lgbm = mean_squared_error(target_test, pred_lgbm, squared=False)
print(f'RMSE para LightGBM: {rmse_lgbm}')

RMSE para LightGBM: 2118.8790340552796


Como podemos visualizar, el modelo de regresión lineal es el que lleva menos tiempo, haciendolo ver el más efectivo hasta ese punto, sin embargo al realizar la evaluación con el conjunto de test se puede observar que el modelo lightgbm obtuvo mejores resultados, por lo tanto, el modelo más recomendable a utilizar para esta aplicación sería LightGBM.

## Conclusiones

Como se puede visualizar a lo largo del proyecto, es muy importante realizar los pasos adecuados, realizar la preparación, entrenamiento, validación, evaluaciones de los modelos, así como también es importante realizar pruebas con diferentes y varios modelos para ver cual es el adecuado, puesto que en un análisis salga con buenos resultados, en otro tipo de análisis arroje resultado distinto, con en el caso que se sucitó en el presente proyecto, que al analizar los tiempos de respuesta en entrenamiento nos arroja resultados, resaltando ser mejor la regresión lineal y catboost, sin embargo, al analizar el RMSE, la calidad del modelo, nos arroja mejores resultados el modelo LightGBM. Lo que se busca es obtener en un modelo y en las predicciones que este realice es calidad, más que un buen tiempo de entrenamiento, por lo que se decidió seleccionar el modelo que arrojará mejores resultados en el RMSE.

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [x]  El código no tiene errores- [x]  Las celdas con el código han sido colocadas en orden de ejecución- [x]  Los datos han sido descargados y preparados- [x]  Los modelos han sido entrenados
- [x]  Se realizó el análisis de velocidad y calidad de los modelos